# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [125]:

# Installs all required packages
# !pip install -r requirements.txt

!pip install requests~=2.32.3
!pip install matplotlib~=3.10.1
!pip install PyInputPlus~=0.2.12
!pip install python-dotenv~=1.1.0
!pip install numpy~=2.2.5
!pip install ipython~=9.2.0
!pip install ipywidgets~=8.1.7


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


## 📦 Setup and Configuration
Import required packages and setup environment.

In [126]:
import math
import os
from datetime import datetime, timedelta

import ipywidgets as widgets
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import requests
from IPython.display import display
from dotenv import load_dotenv
from hands_on_ai.chat import get_response
from requests import HTTPError

# Load environment variables from .env file
load_dotenv()

api_key = os.environ.get('OPENWEATHER_API_KEY')

# Add any other setup code here
if os.environ.get('OPENWEATHER_API_KEY') is None:
    print("Warning: OPENWEATHER_API_KEY not set. Please set it in your environment variables.")

    # Uncomment and replace with your actual API key if you don't want to use .env file
    # os.environ['OPENWEATHER_API_KEY'] = 'your_api_key_here'



## 🌤️ Weather Data Functions

In [127]:
def get_weather_data(location, forecast_day=5):
    """
    Fetch weather data for a given location and forecast day.

    Args:
        location (str): The location to fetch weather data for.
        forecast_day (int): The number of days to forecast (default is 5).

    Returns:
        dict: Weather data for the specified location and forecast day.
    """

    # Use OpenWeatherMap API to get weather details from location
    try:
        # openweathermap forecast provides 3-hourly data
        # for 5 days, so we need to convert it to daily data

        if forecast_day < 1:
            raise ValueError("Forecast day must be at least 1.")
        if forecast_day > 5:
            raise ValueError("Forecast day must be at most 5.")

        forecast_day *= 8  # 3-hourly data for 5 days

        weather_data_response = requests.get(f'https://api.openweathermap.org/data/2.5/forecast?q={location},au'
                                             f'&appid={api_key}&cnt={forecast_day}&units=metric')

        # Raise an error for bad responses
        if weather_data_response.status_code != 200:
            if weather_data_response.status_code == 404:
                raise HTTPError("Location not found. Please check the location name and try again.")
            else:
                raise HTTPError(f"Error fetching weather data: {weather_data_response.status_code}")

        # Parse the JSON response
        weather_data = weather_data_response.json()

        # Check if the response contains weather data
        if 'list' not in weather_data:
            raise ValueError("No weather data found for the specified location.")

        return weather_data
    except ValueError as err:
        print(err)
        return None
    except HTTPError as err:
        print(err)
        return None

## 📊 Visualisation Functions

In [128]:
def create_temperature_visualisation(weather_data, output_type='display', forecast_days=None):
    """
    Create visualisation of temperature data.

    Args:
        :param weather_data: The processed weather data
        :param output_type: Either 'display' to show in notebook or 'figure' to return the figure
        :param forecast_days: Number of forecast days to adjust x-axis formatting

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook

    """
    # Extract temperature data from the weather data
    try:

        days = [day['main'] for day in weather_data['list']]
        dates = [datetime.fromtimestamp(day['dt']) for day in weather_data['list']]
        temperature = [day['temp'] for day in days]

        for i in range(len(temperature)):
            temperature[i] = math.floor(temperature[i])

        # Create a plot
        fig, ax = plt.subplots(figsize=(12, 6))

        # Add day shading
        unique_days = sorted(set([d.date() for d in dates]))
        for i, day in enumerate(unique_days):
            day_start = datetime.combine(day, datetime.min.time())
            day_end = day_start + timedelta(days=1)
            ax.axvspan(day_start, day_end, color='lightgrey' if i % 2 == 0 else 'white',
                       alpha=0.2, zorder=0)

        # Add padding to the x-axis limits
        min_date = min(dates)
        max_date = max(dates)
        padding = timedelta(hours=3)  # Adjust the amount of padding as needed
        ax.set_xlim(min_date - padding, max_date + padding)

        # Check if we need to hide day names based on forecast days
        if forecast_days is not None and forecast_days >= 4:
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m\n%H:%M'))  # Hide day name
        else:
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%a, %d/%m\n%H:%M'))  # Show day name

        ax.xaxis.set_major_locator(mdates.HourLocator(interval=6))
        plt.xticks(rotation=0, ha='center', fontsize=10)
        ax.plot(dates, temperature, marker='o', color='royalblue')
        ax.set_title(f'Temperature Forecast for {weather_data["city"]["name"]}')
        ax.set_ylabel('Temperature (°C)', fontsize=12)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.grid(axis='y', linestyle='--', alpha=0.5)

        if output_type == 'display':
            plt.tight_layout()
            plt.show()
            return None
        else:
            return fig  # Return the figure object
    except KeyError as e:
        print(f"Error processing weather data: {e}")
        return None


def create_precipitation_visualisation(weather_data, output_type='display', forecast_days=None):
    """
    Create visualisation of precipitation data.

    Args:
        weather_data (dict): The processed weather data
        output_type (str): Either 'display' to show in notebook or 'figure' to return the figure

    Returns:
        If output_type is 'figure', returns the matplotlib figure object
        Otherwise, displays the visualisation in the notebook
    """
    # Extract precipitation data from the weather data
    try:
        dates = [datetime.fromtimestamp(day['dt']) for day in weather_data['list']]

        # Extract precipitation data (mm)
        precipitation = []
        for day in weather_data['list']:
            # Get rain amount for 3h period (if exists)
            if 'rain' in day and '3h' in day['rain']:
                precipitation.append(day['rain']['3h'])
            else:
                # No rain in this period
                precipitation.append(0)

        # Create a plot
        fig, ax = plt.subplots(figsize=(12, 6))

        # Add day shading
        unique_days = sorted(set([d.date() for d in dates]))
        for i, day in enumerate(unique_days):
            day_start = datetime.combine(day, datetime.min.time())
            day_end = day_start + timedelta(days=1)
            ax.axvspan(day_start, day_end, color='lightgrey' if i % 2 == 0 else 'white',
                       alpha=0.2, zorder=0)

        # Add padding to the x-axis limits
        min_date = min(dates)
        max_date = max(dates)
        padding = timedelta(hours=3)  # Adjust the amount of padding as needed
        ax.set_xlim(min_date - padding, max_date + padding)
        ax.xaxis.set_major_locator(mdates.HourLocator(interval=6))

        # Check if we need to hide day names based on forecast days
        if forecast_days is not None and forecast_days >= 4:
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%d/%m\n%H:%M'))  # Hide day name
        else:
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%a, %d/%m\n%H:%M'))  # Show day name

        plt.xticks(rotation=0, ha='center', fontsize=10)

        # Plot precipitation as bar chart
        ax.bar(dates, precipitation, width=0.1, color='skyblue', alpha=0.7)

        ax.set_title(f'Precipitation Forecast for {weather_data["city"]["name"]}')
        ax.set_ylabel('Precipitation (mm)', fontsize=12)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.grid(axis='y', linestyle='--', alpha=0.5)

        if output_type == 'display':
            plt.tight_layout()
            plt.show()
            return None
        else:
            return fig  # Return the figure object
    except KeyError as e:
        print(f"Error processing weather data: {e}")
        return None

## 🤖 Natural Language Processing

In [133]:
def parse_weather_question(question):
    """
    Parse the weather question and extract relevant information using hands-on-ai.

    Args:
        question (str): The weather question to parse.

    Returns:
        dict: Parsed information from the question.
    """
    # Prompt to extract structured information from the question
    prompt = f"""
    Extract the following information from this weather question: "{question}"

    1. Location (default to Perth if not specified)
    2. Time period (today, tomorrow, this week, etc. - default to today if not specified)
    3. Weather attribute (temperature or rain, etc. - default to temperature if not specified)

    Return as a JSON object with these fields: location, time_period, attribute
    """

    try:
        if question:
            print(f"Parsing question: {question}")
        else:
            print("No question provided, using default values.")

        # Get structured response from LLM
        response = get_response(prompt=prompt, model="llama3")
        if not response:
            print("No response from the model, using default values.")
            return {"location": "Perth", "time_period": "today", "attribute": "temperature"}

        import json
        import re

        # Try to extract JSON from the response
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            parsed = json.loads(json_match.group(0))
            # Set defaults for missing values
            parsed.setdefault("location", "Perth")
            parsed.setdefault("time_period", "today")
            parsed.setdefault("attribute", "temperature")

            return parsed
        else:
            print("No valid JSON found in the response, using default values.")
            return {"location": "Perth", "time_period": "today", "attribute": "temperature"}

    except Exception as e:
        print(f"Error parsing question: {e}")

        # Fallback to default values if parsing fails
        print("Using default values for location, time period, and attribute.")
        return {"location": "Perth", "time_period": "today", "attribute": "temperature"}


def generate_weather_response(parsed_question):
    """
    Generate a weather response based on the parsed question.
    Args:
        parsed_question (dict): Parsed information from the question.
    Returns:
        str: Generated weather response.
    """
    location = parsed_question.get("location", "Perth")
    time_period = parsed_question.get("time_period", "today").lower()
    attribute = parsed_question.get("attribute", "temperature").lower()

    match time_period:
        case "today":
            generated_forecast_day = 1
        case "tomorrow":
            generated_forecast_day = 2
        case "this week":
            generated_forecast_day = 5
        case _:
            # Default to 3 days if not specified
            print(f"Time period '{time_period}' not supported, defaulting to 3 days.")
            generated_forecast_day = 3

    _response = get_weather_data(location, forecast_day=generated_forecast_day)
    if _response is None:
        return f"Sorry, I couldn't fetch the weather data for {location}."
    if attribute == "temperature":
        return create_temperature_visualisation(_response, output_type='figure', forecast_days=generated_forecast_day)
    elif attribute == "precipitation" or attribute == "rain":
        return create_precipitation_visualisation(_response, output_type='figure', forecast_days=generated_forecast_day)
    else:
        return f"Sorry, I don't have information on {attribute} for {location}."

## 🧭 User Interface

In [130]:
def create_weather_app():
    # Create main container for all outputs
    main_output = widgets.Output()

    # Create view containers
    main_menu_view = widgets.VBox()
    weather_widget_view = widgets.VBox()
    ai_mode_view = widgets.VBox()

    # main menu view
    weather_button = widgets.Button(description='Widget Mode')
    ai_button = widgets.Button(description='AI Query Mode')
    main_menu_view.children = [
        widgets.HTML("<h2>WeatherWise</h2>"),
        widgets.HTML("<p>Select a mode:</p>"),
        weather_button,
        ai_button
    ]

    # weather widget view
    location = widgets.Text(description='Location:', value='Perth')
    forecast_day = widgets.IntSlider(min=1, max=5, description='Forecast Days:', value=3)

    # Graph selection using checkboxes
    show_temp = widgets.Checkbox(value=True, description='Temperature Graph')
    show_precip = widgets.Checkbox(value=False, description='Precipitation Graph')
    graph_options = widgets.HBox([show_temp, show_precip])

    back_button_weather = widgets.Button(description='Back to Main Menu')
    generate_button = widgets.Button(description='Generate Forecast')
    graphs_output = widgets.Output()

    weather_widget_view.children = [
        widgets.HTML("<h2>Weather Forecast</h2>"),
        location, forecast_day,
        widgets.HTML("<p>Select graphs to display:</p>"),
        graph_options,
        widgets.HBox([generate_button, back_button_weather]),
        graphs_output
    ]

    # ai mode view
    query_text = widgets.Text(description='Question:', placeholder='Ask about weather...')
    query_button = widgets.Button(description='Ask')
    ai_output = widgets.Output()
    back_button_ai = widgets.Button(description='Back to Main Menu')

    ai_mode_view.children = [
        widgets.HTML("<h2>AI Weather Assistant</h2>"),
        query_text,
        widgets.HBox([query_button, back_button_ai]),
        ai_output
    ]

    # navigation functions
    def show_main_menu(_):
        with main_output:
            main_output.clear_output()
            display(main_menu_view)

    def show_weather_widgets(_):
        with main_output:
            main_output.clear_output()
            display(weather_widget_view)

    def show_ai_mode(_):
        with main_output:
            main_output.clear_output()
            display(ai_mode_view)

    # make the weather widgets
    def generate_weather(_):
        with graphs_output:
            graphs_output.clear_output()
            weather_data = get_weather_data(location.value, forecast_day.value)

            if weather_data:
                if show_temp.value:
                    create_temperature_visualisation(weather_data,
                                                     output_type='display', forecast_days=forecast_day.value)

                if show_precip.value:
                    create_precipitation_visualisation(weather_data, output_type='display',
                                                       forecast_days=forecast_day.value)

                if not (show_temp.value or show_precip.value):
                    print("Please select at least one graph type")
            else:
                print(f"Failed to fetch weather data for {location.value}.")

    # ai response generation
    def generate_ai_response(_):
        with ai_output:
            ai_output.clear_output()
            question = query_text.value
            parsed = parse_weather_question(question)
            response = generate_weather_response(parsed)
            if isinstance(response, str):
                print(response)
            elif isinstance(response, plt.Figure):
                display(response)  # Display the figure in the output
            else:
                print("Unexpected response type from weather generation.")

    # Connect button callbacks
    weather_button.on_click(show_weather_widgets)
    ai_button.on_click(show_ai_mode)
    back_button_weather.on_click(show_main_menu)
    back_button_ai.on_click(show_main_menu)
    generate_button.on_click(generate_weather)
    query_button.on_click(generate_ai_response)

    # Start with main menu
    show_main_menu(None)
    return main_output


## 🧩 Main Application Logic

In [131]:
def main():
    display(create_weather_app())

## 🧪 Testing and Examples

In [134]:
main()

Output()